In [1]:
import pandas as pd
df = pd.read_csv("master.csv")


df.drop('country-year', inplace=True,axis=1)
df.drop('year', inplace=True,axis=1)#year is technically a derived feature of age and generation
#But year is a precise value.  It can't be exactly derived from age and generation
#But if I don't drop year, model performance is worse
df['sex'].replace(['male','female'],[1,0],inplace=True)
print(df.isna().sum())
df

country                   0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64


,country,sex,age,suicides_no,population,suicides/100k pop,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1,15-24 years,21,312900,6.71,NaN,"2,156,624,900",796,Generation X
1,Albania,1,35-54 years,16,308000,5.19,NaN,"2,156,624,900",796,Silent
2,Albania,0,15-24 years,14,289700,4.83,NaN,"2,156,624,900",796,Generation X
3,Albania,1,75+ years,1,21800,4.59,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1,25-34 years,9,274300,3.28,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,0,35-54 years,107,3620833,2.96,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,0,75+ years,9,348465,2.58,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,1,5-14 years,60,2762158,2.17,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,0,5-14 years,44,2631600,1.67,0.675,"63,067,077,179",2309,Generation Z


In [2]:



for i in range(len(df[' gdp_for_year ($) '])):
    df[' gdp_for_year ($) '][i] = float(df[' gdp_for_year ($) '][i].replace(',',''))



df.drop('suicides_no', inplace=True,axis=1)

df.drop('gdp_per_capita ($)', inplace=True,axis=1)



C:\Users\Charles\AppData\Local\Temp\ipykernel_30408\77645164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[' gdp_for_year ($) '][i] = float(df[' gdp_for_year ($) '][i].replace(',',''))


In [3]:
#df['HDI for year'].fillna(value=df['HDI for year'].mean(),inplace=True)
#df.drop('HDI for year', inplace=True,axis=1)
df.dropna(inplace=True)
df = df.reset_index(drop=True)
#dropping the feature vs replacing null with mean seems to have little effect on model performance
#this is probably because a feature where a majority of null values are sent to some point makes that feature
#as useless as if it wasn't even there in linear regression
# because it will end up getting a low magnitude coefficient in the algorithm 
#But, dropping data points with null values has the best performance.
#So, HDI is probably a useful feature, but dropping this many data points is concerning
#Perhaps there is a more sophisticated way of dealing with all these nulls

In [4]:
def encode_onehot(_df, _f):
    _df2 = pd.get_dummies(_df[_f], prefix='', prefix_sep='').groupby(level=0, axis=1).max().add_prefix(_f+' - ')
    _df3 = pd.concat([_df, _df2], axis=1)
    _df3 = _df3.drop([_f], axis=1)
    return _df3

In [5]:
df = encode_onehot(df,'country')
#df.drop('country', inplace=True,axis=1)
#one hot encoding countries has much better model performance

In [6]:
df2 = encode_onehot(df,'age')


In [7]:
df2 = encode_onehot(df2,'generation')
df2

,sex,population,suicides/100k pop,HDI for year,gdp_for_year ($),country - Albania,country - Antigua and Barbuda,country - Argentina,country - Armenia,country - Australia,...,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent
0,1,232900,5.58,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,178000,5.06,0.619,2424499009.0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,40800,4.90,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,0,283500,4.59,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,241200,4.56,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8359,0,3620833,2.96,0.675,63067077179.0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
8360,0,348465,2.58,0.675,63067077179.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
8361,1,2762158,2.17,0.675,63067077179.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
8362,0,2631600,1.67,0.675,63067077179.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [8]:
df2

,sex,population,suicides/100k pop,HDI for year,gdp_for_year ($),country - Albania,country - Antigua and Barbuda,country - Argentina,country - Armenia,country - Australia,...,age - 35-54 years,age - 5-14 years,age - 55-74 years,age - 75+ years,generation - Boomers,generation - G.I. Generation,generation - Generation X,generation - Generation Z,generation - Millenials,generation - Silent
0,1,232900,5.58,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,178000,5.06,0.619,2424499009.0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,40800,4.90,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,0,283500,4.59,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,241200,4.56,0.619,2424499009.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8359,0,3620833,2.96,0.675,63067077179.0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
8360,0,348465,2.58,0.675,63067077179.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
8361,1,2762158,2.17,0.675,63067077179.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
8362,0,2631600,1.67,0.675,63067077179.0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [9]:
list(df2)

['sex',
 'population',
 'suicides/100k pop',
 'HDI for year',
 ' gdp_for_year ($) ',
 'country - Albania',
 'country - Antigua and Barbuda',
 'country - Argentina',
 'country - Armenia',
 'country - Australia',
 'country - Austria',
 'country - Azerbaijan',
 'country - Bahamas',
 'country - Bahrain',
 'country - Barbados',
 'country - Belarus',
 'country - Belgium',
 'country - Belize',
 'country - Bosnia and Herzegovina',
 'country - Brazil',
 'country - Bulgaria',
 'country - Canada',
 'country - Chile',
 'country - Colombia',
 'country - Costa Rica',
 'country - Croatia',
 'country - Cuba',
 'country - Cyprus',
 'country - Czech Republic',
 'country - Denmark',
 'country - Ecuador',
 'country - El Salvador',
 'country - Estonia',
 'country - Fiji',
 'country - Finland',
 'country - France',
 'country - Georgia',
 'country - Germany',
 'country - Greece',
 'country - Grenada',
 'country - Guatemala',
 'country - Guyana',
 'country - Hungary',
 'country - Iceland',
 'country - Ireland

In [10]:
from sklearn.linear_model import LinearRegression
lg = LinearRegression()
y = df2['suicides/100k pop']
X = df2.drop('suicides/100k pop',axis=1)


In [11]:
lg.fit(X,y)

LinearRegression()

In [12]:
import numpy as np
_y = []
_y_pred = []
for i in range(len(X)):
    if X['sex'][i] == 1 and X['age - 15-24 years'][i] == 1 and X['generation - Generation X'][i] == 1:
        _y_pred.append(list(lg.predict([X.iloc[i]]))[0])
        _y.append(y.iloc[i])
#lg.predict(np.array([[0,1,0,0,0,0,0,0,0,1,0,0,0]]))

C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric valu

In [13]:
_y_pred

[6.966059154351264,
 5.543097488607977,
 16.48594571042623,
 16.12151240157344,
 15.248709597483263,
 14.109935931720173,
 8.192373505882031,
 9.242135289382045,
 7.601459657682881,
 17.290387666624177,
 16.623920404351903,
 15.993594628256893,
 28.724769741756827,
 27.546164434123956,
 26.710869604115103,
 25.899958038465677,
 5.352820582207642,
 4.181565788576471,
 5.495404237640102,
 7.476395921293985,
 4.902760324830698,
 7.869865290763542,
 7.254388343943486,
 6.6769596226878605,
 5.8303674068708595,
 33.47962939967116,
 27.21754115934658,
 25.960780518555687,
 24.20924336144754,
 23.32130251877487,
 12.65451070439589,
 11.886463781366356,
 11.15648339976164,
 14.1957172990192,
 13.156393455390614,
 11.960321718215035,
 11.087696289936755,
 22.005603026917086,
 21.671482361656274,
 21.628305982042633,
 20.968484359493367,
 16.921176189165173,
 15.987131167865243,
 15.547875063442461,
 15.33000076574637,
 19.101595569482754,
 17.37494843185214,
 16.44767438550599,
 15.3422571919584

In [14]:
def mae(_y, _y_pred):
    return (len(_y)**-1) * np.sum(np.abs(_y_pred-_y))

In [15]:
print("MAE problem1: ",mae(np.array(_y),np.array(_y_pred)))

MAE problem1:  6.607892933743736


In [16]:
print("There are",len(lg.coef_),"regression coefficients.")

There are 106 regression coefficients.


In [17]:
df3 = df.copy()


In [18]:
df3['age'].unique()

array(['25-34 years', '55-74 years', '75+ years', '15-24 years',
       '35-54 years', '5-14 years'], dtype=object)

In [19]:
df3['age'].replace(['15-24 years', '35-54 years', '75+ years', '25-34 years',
       '55-74 years', '5-14 years'],[19.5,44.5,89.5,29.5,64.5,9.5],inplace=True)
#replacing age ranges with their mean, and following pattern of +10,+15,+20,+25 for 75+years range
#maybe it would be better to replace with 2 columns: 1 for low end and one for high end
#sex remains binary
df3

,sex,age,population,suicides/100k pop,HDI for year,gdp_for_year ($),generation,country - Albania,country - Antigua and Barbuda,country - Argentina,...,country - Thailand,country - Trinidad and Tobago,country - Turkey,country - Turkmenistan,country - Ukraine,country - United Arab Emirates,country - United Kingdom,country - United States,country - Uruguay,country - Uzbekistan
0,1,29.5,232900,5.58,0.619,2424499009.0,Generation X,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,64.5,178000,5.06,0.619,2424499009.0,Silent,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,89.5,40800,4.90,0.619,2424499009.0,G.I. Generation,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,19.5,283500,4.59,0.619,2424499009.0,Generation X,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,19.5,241200,4.56,0.619,2424499009.0,Generation X,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8359,0,44.5,3620833,2.96,0.675,63067077179.0,Generation X,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8360,0,89.5,348465,2.58,0.675,63067077179.0,Silent,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8361,1,9.5,2762158,2.17,0.675,63067077179.0,Generation Z,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8362,0,9.5,2631600,1.67,0.675,63067077179.0,Generation Z,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
df3['generation'].unique()

array(['Generation X', 'Silent', 'G.I. Generation', 'Boomers',
       'Millenials', 'Generation Z'], dtype=object)

In [21]:
df3['generation'].replace(['Generation X', 'Silent', 'G.I. Generation', 'Boomers',
       'Millenials', 'Generation Z'],[1972.5,1936.5,1914,1955,1988.5,2004],inplace=True)
#replacing generation with average birth year
df3

,sex,age,population,suicides/100k pop,HDI for year,gdp_for_year ($),generation,country - Albania,country - Antigua and Barbuda,country - Argentina,...,country - Thailand,country - Trinidad and Tobago,country - Turkey,country - Turkmenistan,country - Ukraine,country - United Arab Emirates,country - United Kingdom,country - United States,country - Uruguay,country - Uzbekistan
0,1,29.5,232900,5.58,0.619,2424499009.0,1972.5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,64.5,178000,5.06,0.619,2424499009.0,1936.5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,89.5,40800,4.90,0.619,2424499009.0,1914.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,19.5,283500,4.59,0.619,2424499009.0,1972.5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,19.5,241200,4.56,0.619,2424499009.0,1972.5,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8359,0,44.5,3620833,2.96,0.675,63067077179.0,1972.5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8360,0,89.5,348465,2.58,0.675,63067077179.0,1936.5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8361,1,9.5,2762158,2.17,0.675,63067077179.0,2004.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8362,0,9.5,2631600,1.67,0.675,63067077179.0,2004.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
lg = LinearRegression()
y = df3['suicides/100k pop']
X = df3.drop('suicides/100k pop',axis=1)
lg.fit(X,y)

LinearRegression()

In [23]:
_y = []
_y_pred = []
for i in range(len(X)):
    if X['sex'][i] == 1 and X['age'][i] == 19.5 and X['generation'][i] == 1972.5:
        _y_pred.append(list(lg.predict([X.iloc[i]]))[0])
        _y.append(y.iloc[i])

C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\Charles\anaconda3\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric valu

In [24]:
print("MAE problem1: ",mae(np.array(_y),np.array(_y_pred)))
print("There are",len(lg.coef_),"regression coefficients.")

MAE problem1:  6.043199741062099
There are 96 regression coefficients.


Q3: Yes, the numerical model performed better in terms of MAE by about .7 in all combinations of preprocessing null/HDI, country, and year.

In [25]:
X_2 = X.copy()
for name in list(X_2):
    if 'country' in name:
        X_2.drop(name,axis=1,inplace=True)

X_2


,sex,age,population,HDI for year,gdp_for_year ($),generation
0,1,29.5,232900,0.619,2424499009.0,1972.5
1,1,64.5,178000,0.619,2424499009.0,1936.5
2,0,89.5,40800,0.619,2424499009.0,1914.0
3,0,19.5,283500,0.619,2424499009.0,1972.5
4,1,19.5,241200,0.619,2424499009.0,1972.5
...,...,...,...,...,...,...
8359,0,44.5,3620833,0.675,63067077179.0,1972.5
8360,0,89.5,348465,0.675,63067077179.0,1936.5
8361,1,9.5,2762158,0.675,63067077179.0,2004.0
8362,0,9.5,2631600,0.675,63067077179.0,2004.0


In [26]:
lg.fit(X_2,y)

LinearRegression()

In [27]:
X_0 = pd.DataFrame.from_dict({name:[X_2[name].mean()] for name in list(X_2)})
X_0['sex'] = 1
X_0['age'] =33
X_0['generation'] = 2020
X_0

,sex,age,population,HDI for year,gdp_for_year ($),generation
0,1,33,1.852173e+06,0.776601,5.476639e+11,2020


In [28]:
print("Q4: There will be",list(lg.predict(X_0))[0], "suicides for a 33 year old male generation alpha per 100k.  This was considered a low rate in my week 3 assignment.")

Q4: There will be 9.336659944384337 suicides for a 33 year old male generation alpha per 100k.  This was considered a low rate in my week 3 assignment.


Q5: Regression allows for numerical output.  In order for classification to replicate this, we would have to classify numerical ranges as low, medium, high, etc. Also, I saw that we can use regression on unseen features.

Q6: If a feature is in some way ordinal, like generation, then one-hot encoding this feature will lose that information, because we will get different coefficients on a feature.

Q7: I prefer regression because it can give more precise predictions.  The customer can make their own decisions on how to classify values based on numerical output.